### Notebook to genereate box and whisker plots from TC output binned by cloud classification scheme.

Assumes output is in a single netcdf file on pressure levels.

James Ruppert  
jruppert@ou.edu  
12/8/23

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib
from matplotlib import ticker
import matplotlib.pyplot as plt
import subprocess
import sys
from thermo_functions import theta_virtual, relh, theta_equiv
from cfads_functions import mask_edges
from precip_class import precip_class
import seaborn as sns

#### Main settings

In [2]:
# Number of sample time steps
nt=200 # will be chopped down to max available
nt=12

#### Additional settings and directories

In [3]:
storm = 'haiyan'
# storm = 'maria'

# main = "/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/wrfenkf/"
main = "/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/tc_ens/"
figdir = "/home/jamesrup/figures/tc/ens/"+storm+'/'
datdir2 = 'post/d02/'

# Tests to read and compare
# tests = ['crfon','ncrf']
# if storm == 'haiyan':
#     tests = ['ctl','ncrf36h']
# elif storm == 'maria':
#     # tests = ['ctl','ncrf36h']
#     tests = ['ctl','ncrf48h']
tests = ['ctl']

time_neglect=12 # time steps from start to neglect

# Members
nmem = 10 # number of ensemble members (1-5 have NCRF)
# nmem = 5
nmem = 2
enstag = str(nmem)

In [4]:
# Ensemble member info
memb0=1 # Starting member to read
nums=np.arange(memb0,nmem+memb0,1); nums=nums.astype(str)
nustr = np.char.zfill(nums, 2)
memb_all=np.char.add('memb_',nustr)

##### Get dimensions

datdir = main+storm+'/'+memb_all[0]+'/'+tests[0]+'/'+datdir2
datdir3d = datdir #+'v2/'
varfil_main = Dataset(datdir3d+'T.nc')
nz = varfil_main.dimensions['level'].size
nx1 = varfil_main.dimensions['lat'].size
nx2 = varfil_main.dimensions['lon'].size
nt_data = varfil_main.dimensions['time'].size
nt=np.min([nt,nt_data-time_neglect])
pres = varfil_main.variables['pres'][:] # hPa
dp = (pres[1]-pres[0])*1e2 # Pa
varfil_main.close()

# process = subprocess.Popen(['ls '+main+storm+'/'+memb_all[0]+'/'+tests[0]+'/wrfout_d02_*'],shell=True,
#     stdout=subprocess.PIPE,universal_newlines=True)
# output = process.stdout.readline()
# wrffil = output.strip() #[3]
# varfil_main = Dataset(wrffil)
# lat = varfil_main.variables['XLAT'][:][0] # deg
# lon = varfil_main.variables['XLONG'][:][0] # deg
# varfil_main.close()

#### NetCDF variable read functions

In [5]:
def var_read_3d(datdir,varname,t0,t1):
    varfil_main = Dataset(datdir+varname+'.nc')
    var = varfil_main.variables[varname][t0:t1,:,:,:]
    varfil_main.close()
    mask_edges(var)
    return var
def var_read_2d(datdir,varname,t0,t1):
    varfil_main = Dataset(datdir+varname+'.nc')
    var = varfil_main.variables[varname][t0:t1,:,:,:]
    varfil_main.close()
    mask_edges(var)
    return var

In [6]:
# For MSE variance terms

def read_mse(datdir,t0,t1):
    varfil_main = Dataset(datdir+'mse_diag.nc')
    mse = varfil_main.variables['mse_int'][t0:t1,:,:] # J/m2
    varfil_main.close()
    return mse

def read_vmf(datdir,t0,t1):
    varfil_main = Dataset(datdir+'mse_diag.nc')
    vmfu = varfil_main.variables['vmfu'][t0:t1,:,:] # kg/m/s
    vmfd = varfil_main.variables['vmfd'][t0:t1,:,:] # kg/m/s
    varfil_main.close()
    return vmfu, vmfd

def read_lwnet(datdir,t0,t1):
    lw_t = var_read_2d(datdir,'LWUPT',t0,t1) - var_read_2d(datdir,'LWDNT',t0,t1) # W/m2
    lw_b = var_read_2d(datdir,'LWUPB',t0,t1) - var_read_2d(datdir,'LWDNB',t0,t1) # W/m2
    lwnet = lw_b - lw_t
    return lwnet

def read_lwnetc(datdir,t0,t1):
    lw_t = var_read_2d(datdir,'LWUPTC',t0,t1) - var_read_2d(datdir,'LWDNTC',t0,t1) # W/m2
    lw_b = var_read_2d(datdir,'LWUPBC',t0,t1) - var_read_2d(datdir,'LWDNBC',t0,t1) # W/m2
    lwnetc = lw_b - lw_t
    return lwnetc

#### Main loops and compositing

In [7]:
# Main read loops for 3D (dependent) variables

# Arrays to save variables
ntest=len(tests)
strat_all  = np.ma.zeros((ntest,nmem,nt,nx1,nx2))
cwv_all    = np.ma.zeros((ntest,nmem,nt,nx1,nx2))
lwacre_all = np.ma.zeros((ntest,nmem,nt,nx1,nx2))
lwnet_all  = np.ma.zeros((ntest,nmem,nt,nx1,nx2))
vmfu_all   = np.ma.zeros((ntest,nmem,nt,nx1,nx2))
vmfd_all   = np.ma.zeros((ntest,nmem,nt,nx1,nx2))
# mse_all = np.ma.zeros((ntest,nmem,nt,nx1,nx2))
# mselw_all = np.ma.zeros((ntest,nmem,nt,nx1,nx2))

for ktest in range(ntest):

    test_str=tests[ktest]

    # This has been tested for corresponding time steps:
    #   t0=37,1 are the first divergent time steps in CTL,NCRF
    #   t0=25,1 are the first divergent time steps in NCRF,CRFON
    # if test_str == 'ctl':
    #     if tests[1] == 'ncrf36h':
    #         t0=36
    #     elif tests[1] == 'ncrf48h':
    #         t0=48
    # elif test_str == 'ncrf36h':
    #     t0=t0_test
    # elif test_str == 'ncrf48h':
    #     t0=t0_test
    # elif test_str == 'crfon':
    #     t0=0
    t0=time_neglect # neglect the first 12 time steps
    t1=t0+nt

    # t0+=1 # add one time step since NCRF(t=0) = CTL
    # t1 = t0+nt

    print('Running test: ',test_str)

    # Loop over ensemble members

    for imemb in range(nmem):
    
        print('Running imemb: ',memb_all[imemb])
    
        datdir = main+storm+'/'+memb_all[imemb]+'/'+test_str+'/'+datdir2
        datdir3d = datdir #+'v2/'
        print(datdir)

        # Stratiform ID
        varfil_main = Dataset(datdir+'q_int.nc')
        q_int = varfil_main.variables['q_int'][:,t0:t1,:,:] # 'nq','nt','nx1','nx2',
        varfil_main.close()
        strat = precip_class(q_int)

        # MSE variance
        # mse = read_mse(datdir,t0,t1) # J/m2

        # lwnet
        lwnet = read_lwnet(datdir,t0,t1) # W/m2
        lwnetc = read_lwnetc(datdir,t0,t1) # W/m2
        lwacre = lwnet - lwnetc # W/m2

        # VMF
        vmfu, vmfd = read_vmf(datdir,t0,t1) # kg/m2/s

        # CWV
        varname='PW'
        cwv = var_read_2d(datdir3d,varname,t0,t1) # mm
        # ddtq = np.gradient(lwnet, axis=0) # mm/hr

        ### Process and save variable ##############################################

        # Save ens member
        strat_all[ktest,imemb,:,:,:]  = strat
        lwacre_all[ktest,imemb,:,:,:] = lwacre[:,0,:,:]
        lwnet_all[ktest,imemb,:,:,:]  = lwnet[:,0,:,:]
        cwv_all[ktest,imemb,:,:,:]    = cwv[:,0,:,:]
        vmfu_all[ktest,imemb,:,:,:]   = vmfu
        vmfd_all[ktest,imemb,:,:,:]   = vmfd

Running test:  ctl
Running imemb:  memb_01
/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/tc_ens/haiyan/memb_01/ctl/post/d02/
Running imemb:  memb_02
/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/tc_ens/haiyan/memb_02/ctl/post/d02/


In [8]:
vmf_all = vmfu_all + vmfd_all

### Run binning

In [9]:
def get_kstrat_cells(var_in, strat):
    indices = (strat == 0).nonzero()
    var_np = var_in[indices]

    indices = (strat == 1).nonzero()
    var_dc = var_in[indices]

    indices = (strat == 2).nonzero()
    var_cg = var_in[indices]

    indices = (strat == 3).nonzero()
    var_sh = var_in[indices]

    indices = (strat == 4).nonzero()
    var_st = var_in[indices]

    indices = (strat == 5).nonzero()
    var_an = var_in[indices]

    return var_np, var_dc, var_cg, var_sh, var_st, var_an

In [10]:
for ktest in range(ntest):
    cwv_np, cwv_dc, cwv_cg, cwv_sh, cwv_st, cwv_an = get_kstrat_cells(cwv_all[ktest], strat_all[ktest])
    lwnet_np, lwnet_dc, lwnet_cg, lwnet_sh, lwnet_st, lwnet_an = get_kstrat_cells(lwnet_all[ktest], strat_all[ktest])
    lwacre_np, lwacre_dc, lwacre_cg, lwacre_sh, lwacre_st, lwacre_an = get_kstrat_cells(lwacre_all[ktest], strat_all[ktest])
    vmfu_np, vmfu_dc, vmfu_cg, vmfu_sh, vmfu_st, vmfu_an = get_kstrat_cells(vmfu_all[ktest], strat_all[ktest])
    vmfd_np, vmfd_dc, vmfd_cg, vmfd_sh, vmfd_st, vmfd_an = get_kstrat_cells(vmfd_all[ktest], strat_all[ktest])
    vmf_np, vmf_dc, vmf_cg, vmf_sh, vmf_st, vmf_an = get_kstrat_cells(vmf_all[ktest], strat_all[ktest])

---
### Plotting routines

In [44]:
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

In [59]:
c_name = ['Deep\nConvective', 'Congestus', 'Shallow\nConvective', 'Stratiform', 'Anvil']
cmap = ['teal', 'plum', 'darkorange', 'gold', 'cornflowerblue']
sns.set_palette(cmap)

fig = plt.figure(figsize=(8,6),dpi=500)
fig.set_facecolor('white')
ax = fig.subplots(nrows=1, ncols=1)
sns.boxplot([plt_dc,plt_con,plt_sc,plt_strat,plt_a], showmeans=True, meanprops={"marker":"o",
                    "markerfacecolor":"white", 
                    "markeredgecolor":"black",
                    "markersize":"6"})
ax.set_xticklabels(c_name)
# plt.xticks(labels=c_name)
plt.ylabel('[day$^{-1}$]', weight='bold')
plt.title("d) Class Averaged $ \mathbf{\hat{h}' NETLW'}$", weight='bold')